In [1]:
import sys
import os

parent_path = os.path.abspath("..")  # Get absolute path of parent directory
if parent_path not in sys.path:  
    sys.path.append(parent_path)  # Add only if it's not already there
    
from dotenv import load_dotenv
load_dotenv("../.env")


import pandas as pd
import chromadb
import openai

from src.models.chromadb_get_collection import *  
from src.models.embedding import *  

### Read Excel file 

In [ ]:
question_answer_df = pd.read_excel("../data/question_answer/QA.xlsx")

print(question_answer_df)

print("Excel Length: ", len(question_answer_df))

   no                    base_question                 answer  \
0   1   Tại sao cần bón phân cho cây ?        Để cây tươi tốt   
1   2  Cây nhãn chăm sóc như thế nào ?  Bón phân thường xuyên   

                                    variant_question  
0  Lí do cần bón phân cho cây \nTại sao cây cần p...  
1  Chăm sóc cây nhãn ra sao ?\nLàm sao chăm sóc c...  
Excel Length:  2


### Get ChromaDB Collection

In [3]:
collection_path     = "../data/vector_database"
collection_name     = os.getenv("CHROMADB_COLLECTION_NAME")
embedding_func      = define_embedding_function( api_key = os.getenv("OPENAI_API_KEY"), model_name = os.getenv("EMBEDDING_MODEL"))
similarity_method   = os.getenv("SIMILARITY_METHOD")

In [4]:
chromadb_collection = get_chroma_collection(collection_path, collection_name, embedding_func, similarity_method)

### Load Questions into ChromaDB
- With index in metadata and index for tracking

In [5]:
list_base_question  = list(question_answer_df["base_question"])
list_question_no    = list(question_answer_df["no"])

In [6]:
chromadb_collection.add(
    documents=list_base_question,
    metadatas = [{"video": f"{i}.mp4"} for i in list_question_no],
    ids = [str(i) for i in list_question_no]
)

### Update question in ChromaDB using index

In [ ]:
chromadb_collection.update(
    ids=[str(i) for i in list_question_no],
    metadatas=[{"video": f"{i}.mp4"} for i in list_question_no]
)

### Query Similar Question in ChromaDB

In [ ]:
results = chromadb_collection.query(
    query_texts=["Làm sao để chăm sóc cây ăn trái?"], # Chroma will embed this for you
    n_results=1 # how many results to return
)

print(results)